# Parquet files 1: Single Parquet files
By the end of this lecture you will be able to:
- read from a Parquet file
- use query optimisation to read a subset of the data
- get the schema of a Parquet file
- write a Parquet file with compression
- write a Parquet file that is larger than memory

## What is a Parquet file?
A Parquet file is:
- a *binary* file where data is ordered in columns rather than rows
- each column has a name and a dtype
- each column can be compressed separately with automatic dictionary encoding

The Apache Parquet and Apache Arrow projects evolved together as columnar formats where Apache Parquet is the format for the data on disk and Apache Arrow is the format for the data in memory.

Compared to CSV a Parquet file:
- is faster to read and write than a CSV file
- takes less space on disk, especially once compression is applied
- allows Polars to select which columns to read without parsing the full dataset
- preserves the dtypes of columns

*I use Parquet files whenever possible for my data engineering pipelines* with the exception of small files that I want to open to read manually

In [ ]:
from pathlib import Path

import polars as pl

## Creating a Titanic Parquet file
We begin by creating a Parquet file from the Titanic CSV file

In [ ]:
csv_file = "../data/titanic.csv"

We create the Parquet Titanic directory in the `data_files/parquet` sub-directory of the `io` sub-directory

In [ ]:
parquet_file_path = Path("data_files/parquet/titanic")
if not parquet_file_path.exists():
    parquet_file_path.mkdir(parents=True,exist_ok=True)

Now we set the path that we will write the Parquet file to

In [ ]:
parquet_file = "data_files/parquet/titanic/titanic.parquet"

We read the CSV and write to the Parquet path

In [ ]:
pl.read_csv(csv_file).write_parquet(parquet_file)

## Reading a Parquet file
We read the Parquet file to a `DataFrame`

In [ ]:
df = pl.read_parquet(parquet_file)
df.head(3)

As a Parquet file stores the schema as metadata we can get the schema of a Parquet file without having to read any data.

In Polars we can use the `read_parquet_schema` function for this

In [ ]:
pl.read_parquet_schema(parquet_file)

We see that the dtypes are preserved in a Parquet file (unlike a CSV file where all data is converted to text)

We can select a subset of columns to read from a Parquet file with the `columns` argument

In [ ]:
(
    pl.read_parquet(
        parquet_file,
        columns=["Pclass","Name"]
    )
    .head(3)
)

When we work in lazy mode in Polars the query optimiser detects when only a subset of columns must be read automatically

In [ ]:
print(
    pl.scan_parquet(parquet_file)
    .select(["Pclass","Name"])
    .explain()
)

We can also specify a smaller number of rows that we want to read with `n_rows`

In [ ]:
(
    pl.read_parquet(
        parquet_file,
        n_rows=2
    )
)

If we are running out of memory when reading a Parquet file we can specify `low_memory = True`. This can help to reduce peak memory usage at the expense of a longer load time

In [ ]:
(
    pl.read_parquet(
        parquet_file,
        low_memory=True
    )
    .head(2)
)

Polars reads the Parquet file in multiple threads into different chunks of memory. By default Polars then combines all the chunks into a single chunk in parallel. With the `low_memory=True` argument Polars reduces peak memory usage by not doing this recombination in parallel.

Using `low_memory = True` will not help if the ultimate `DataFrame` does not fit in memory. In this case using `streaming` in lazy mode is the best option

## Writing a Parquet file
When we write a Parquet file we can specify compression options. I recommend using `zstd` in most cases for a good balance of compressed file size on disk and read time into memory. The `lz4` option is an alternative when faster reading and writing is preferred.

In [ ]:
df.write_parquet(parquet_file,compression="zstd")

We can also adjust the degree of compression with `compression_level`. The range of values depends on the compression scheme chosen - see the docstrings for details.

A Parquet file internally is broken into groups of rows (called row groups). Parquet files can store simple statistics of the data in each row group. In a lazy query Polars can use these statistics to determine if only some row groups of the file need to be read.

- We can tell Polars to add the statistics for row groups in the file by adding `statistics=True`
- We can tell Polars how many rows should be in each row_group with the `row_group_size` (set to 512^2 by default)

In [ ]:
df.write_parquet(
    parquet_file,
    compression="zstd",
    statistics=True,
)

One further point to get maximum value from the row groups: if you want to query the Parquet file in a certain way (e.g. pull out  date ranges or pull out certain values from a ID column) then sort the `DataFrame` by that column (or columns) before writing to concentrate the data in the smallest number of row groups inside the file.

Note: writing statistics can be slow for larger files so it is only worth doing if you will get value from faster repeated queries against that file.

If we apply a filter on a **lazy** scan of a Parquet file Polars uses the row groups to reduce how much data must be read - based on on the filter in the `SELECTION` of the query plan below

In [ ]:
print(
    pl.scan_parquet(parquet_file)
    .filter(
        pl.col("PassengerId") < 30
    )
    .explain()
)

### Writing a larger-than-memory Parquet file
We can use streaming to process a larger-than-memory Parquet file (or files) and "sink" (or write) the output to another Parquet file. Whereas normal streaming requires the output to fit into memory the `sink` approach removes this contraint

In [ ]:
sinkparquet_file = "data_files/parquet/titanic/titanic_sink.parquet"
(
    pl.scan_parquet(parquet_file)
    .group_by("Pclass")
    .agg(
        pl.col("PassengerId").count().alias("counts")
    )
    .sink_parquet(sinkparquet_file)
)

The `sink_parquet` approach only requires a `LazyFrame`, the query does not have to begin with a `scan_parquet`. This means we can use it to convert a larger-than-memory CSV file to a Parquet file

In [ ]:
sinkparquet_file = "data_files/parquet/titanic/titanic_sink.parquet"
(
    pl.scan_csv(csv_file)
    .group_by("Pclass")
    .agg(
        pl.col("PassengerId").count().alias("counts")
    )
    .sink_parquet(sinkparquet_file)
)

There is also a `sink_csv` method available: https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.LazyFrame.sink_csv.html

## Exercises
In the exercises you will develop your understanding of:
- read and writing Parquet files
- categorical dtypes in Parquet files
- reading the schema of Parquet files
- reading a subset of Parquet files

### Exercise 1
We will write a new Parquet file for the exercises to this path

In [ ]:
exercise_parquet_file = "data_files/parquet/titanic/titanic_exercise.parquet"

Before we write to this file read the Parquet file created at the start of the notebook to a `DataFrame`. 

Convert the `Sex` column to `pl.Categorical`

In [ ]:
df = (
    pl.read_parquet(parquet_file)
    .with_columns(<blank>)
)
df.head(3)

Write the `DataFrame` with a categorical column to `exercise_parquet_file`

Read the schema of `exercise_parquet_file` to confirm whether Parquet can preserve categorical encodings

Create a lazy query that only reads these columns
```python
["Survived","Pclass","Age","Sex"]
```

## Solutions

### Solution to exercise 1
We will write a new Parquet file for the exercises to this path

In [ ]:
exercise_parquet_file = "data_files/parquet/titanic/titanic_exercise.parquet"

Before we write to this file read the Parquet file created at the start of the notebook to a `DataFrame`. 

Convert the `Sex` column to `pl.Categorical`

In [ ]:
df = (
    pl.read_parquet(parquet_file)
    .with_columns(pl.col("Sex").cast(pl.Categorical))
)
df.head(3)

Write the `DataFrame` with a categorical column to `exercise_parquet_file`

In [ ]:
df.write_parquet(exercise_parquet_file)

Read the schema of `exercise_parquet_file` to confirm whether Parquet can preserve categorical encodings

In [ ]:
pl.read_parquet_schema(exercise_parquet_file)

Create a lazy query that only reads these columns
```python
["Survived","Pclass","Age","Sex"]
```

In [ ]:
(
    pl.scan_parquet(exercise_parquet_file)
    .select(["Survived","Pclass","Age","Sex"])
)

### Solution to exercise 2
This exercise is still in development but you can read through if interested.

Here we compare performance reading a Parquet file with and without statistics

First create a long `DataFrame`

In [ ]:
import numpy as np
N = 3_000_000
df_random = pl.DataFrame(
    np.random.standard_normal((N,3)),
    schema=["a","b","c"]
)
df_random.head()

Now we create a new directory to save this Parquet file to

In [ ]:
random_parquet_file_path = Path("data_files/parquet/big")
if not random_parquet_file_path.exists():
    random_parquet_file_path.mkdir(parents=True,exist_ok=True)

Now we create a file path with and without statistics

In [ ]:
random_parquet_stats = random_parquet_file_path / "stats.parquet"
random_parquet_no_stats = random_parquet_file_path / "no_stats.parquet"

Save the Parquet file with and without statistics to the appropriate file path

In [ ]:
df_random.write_parquet(random_parquet_stats,statistics=True)
df_random.write_parquet(random_parquet_no_stats)

Compare how long the following query needs with and without statistics

In [ ]:
%%timeit -n1 -r3
(
    pl.scan_parquet(random_parquet_stats)
    .filter(
        pl.col("a")<-2
    )
    .collect()
)

In [ ]:
%%timeit -n1 -r3
(
    pl.scan_parquet(random_parquet_no_stats)
    .filter(
        pl.col("a")<-2
    )
    .collect()
)

Now try reading again where we have sorted the data by column `a`

In [ ]:
df_random.sort("a").write_parquet(random_parquet_stats,statistics=True)
df_random.sort("a").write_parquet(random_parquet_no_stats)